In [5]:
import numpy as np
import pandas as pd
import struct
from datetime import datetime, timedelta
import os

# === CONFIGURACIÓN ===
folder_raw_data = 'data_raw'
filename_raw_data = '20-11-2025_T79_T5'  # Cambiar por el archivo deseado
filename_folder_data_out = 'data_csv'
filename = os.path.join(folder_raw_data, filename_raw_data + '.DAT')

acc_scaler = 0.00059841
gyro_scaler = 0.00875

# === CONVERSIÓN DE BYTES ===
def bytes_to_16int(b):
    val = b[0] + (b[1] << 8)
    return val - 65536 if val > 32767 else val

def bytes_to_acc(b):
    return acc_scaler * bytes_to_16int(b)

def bytes_to_gyro(b):
    return gyro_scaler * bytes_to_16int(b)

def bytes_to_32uint(b):
    return struct.unpack('<I', b)[0]

def bytes_to_16uint(b):
    return b[0] + (b[1] << 8)

# === EXTRAER INFO DE BOOTUP ===
def get_time_bootup(byte_array):
    fr_version = byte_array[215:218].tobytes().decode('ascii')
    if fr_version == '1.0':
        rtos_res = bytes_to_32uint(byte_array[244:248])
        offset_secs = bytes_to_32uint(byte_array[260:264]) / rtos_res
        date_str = byte_array[110:180].tobytes().decode('ascii')
        d = date_str.split('date:')[1].split(' ')[0]
        t = date_str.split('time ')[1].split(' UTC')[0]
        dt_utc = f"{d[:4]}-{d[4:6]}-{d[6:]} {t}"
    else:
        rtos_res = bytes_to_32uint(byte_array[244:248])
        offset_secs = bytes_to_32uint(byte_array[268:272]) / rtos_res
        date_str = byte_array[110:180].tobytes().decode('ascii')
        dt_utc = date_str.split('date:')[1].split(' UTC')[0]
    dt = datetime.strptime(dt_utc, '%Y-%m-%d %H:%M:%S')
    return rtos_res, dt, offset_secs

# === PARSER IMU0AG ===
def get_accelGyro(byte_array, dateTime_UTC, t0, t1):
    n = bytes_to_16uint(byte_array[16:18])
    dt = (t1 - t0) / n
    time_list = [(dateTime_UTC + timedelta(seconds=t0 + i * dt)).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3] for i in range(n)]

    gyro_on = bytes_to_16uint(byte_array[34:36])
    gx = []; gy = []; gz = []; ax = []; ay = []; az = []

    if gyro_on:
        for i in range(n):
            idx = 50 + 12 * i
            gx.append(round(bytes_to_gyro(byte_array[idx:idx+2]), 2))
            gy.append(round(bytes_to_gyro(byte_array[idx+2:idx+4]), 2))
            gz.append(round(bytes_to_gyro(byte_array[idx+4:idx+6]), 2))
            ax.append(round(bytes_to_acc(byte_array[idx+6:idx+8]), 4))
            ay.append(round(bytes_to_acc(byte_array[idx+8:idx+10]), 4))
            az.append(round(bytes_to_acc(byte_array[idx+10:idx+12]), 4))
    else:
        for i in range(n):
            idx = 50 + 6 * i
            gx.append(0); gy.append(0); gz.append(0)
            ax.append(round(bytes_to_acc(byte_array[idx:idx+2]), 4))
            ay.append(round(bytes_to_acc(byte_array[idx+2:idx+4]), 4))
            az.append(round(bytes_to_acc(byte_array[idx+4:idx+6]), 4))

    return gx, gy, gz, ax, ay, az, time_list

# === BUSCAR INICIOS DE BLOQUES ===
def find_start_indexes(byte_array):
    idxs = []
    br_locs = np.where(byte_array == 13)[0]
    for i in br_locs:
        if (i+2 < len(byte_array)) and (byte_array[i+1] == 10) and (48 <= byte_array[i+2] <= 122 or byte_array[i+2] == 33):
            idxs.append(i)
    return idxs

# === PARSEAR TODO EL ARCHIVO ===
def get_datacsv_from_file(byte_array, indexes):
    gx=[]; gy=[]; gz=[]; ax=[]; ay=[]; az=[]; a_time=[]
    for i in indexes:
        task_name = byte_array[i+2:i+14].tobytes()
        if task_name.startswith(b'SYSTEM INFO'):
            rtos, utc, offset = get_time_bootup(byte_array[i:i+300])
            t_scaler = 1 / rtos
        elif task_name.startswith(b'IMU0AG'):
            data_size = bytes_to_16uint(byte_array[i+14:i+16])
            t0_raw = bytes_to_32uint(byte_array[i+20:i+24])
            t1_raw = bytes_to_32uint(byte_array[i+24:i+28])
            t0 = t0_raw * t_scaler - offset
            t1 = t1_raw * t_scaler - offset
            gxu, gyu, gzu, axu, ayu, azu, tu = get_accelGyro(byte_array[i:i+data_size], utc, t0, t1)
            gx += gxu; gy += gyu; gz += gzu
            ax += axu; ay += ayu; az += azu
            a_time += tu

    df = pd.DataFrame({
        'dateTime_UTC': a_time,
        'ax': ax, 'ay': ay, 'az': az,
        'gx': gx, 'gy': gy, 'gz': gz
    })
    return df

# === PROCESAR Y GUARDAR ===
byte_array = np.fromfile(filename, dtype=np.uint8)
indexes = find_start_indexes(byte_array)
accel_frame = get_datacsv_from_file(byte_array, indexes)

# Crear carpeta si no existe
if not os.path.exists(filename_folder_data_out):
    os.makedirs(filename_folder_data_out)

# Guardar CSV
filename_data_out = os.path.join(filename_folder_data_out, filename_raw_data + '_accelGyro.csv')
accel_frame.to_csv(filename_data_out, index=False)
print(f"CSV guardado en: {filename_data_out}")


CSV guardado en: data_csv\20-11-2025_T79_T5_accelGyro.csv
